In [3]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from yahooquery import Ticker
import time
from tqdm import tqdm
import numpy as np
import collections
from db_connector import connect_db
from db_connector import engine_db

if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

engine = engine_db()
con = connect_db()

mycursor = con.cursor()

ticker_list = pd.read_sql("""
select * from global_ticker
where date = (select max(date) from global_ticker)
and country = 'United States'
and (`Market Cap` like '%%B' and length(`Market Cap`) > 6
OR (`Market Cap` like '%%T'));    
""", con=engine)

truncate_table = """
truncate table global_fs;
"""

mycursor.execute(truncate_table)
con.commit()

query_fs = """
    insert into global_fs (ticker, date, account, value, freq)
    values (%s,%s,%s,%s,%s) as new
    on duplicate key update
    value = new.value;
"""

error_list = []

for i in tqdm(range(0, len(ticker_list))):

    ticker = ticker_list['Symbol'][i]

    try:

        data = Ticker(ticker)
        
        data_y = data.all_financial_data(frequency = 'a')
        data_y.reset_index(inplace = True)
        data_y = data_y.loc[:, ~data_y.columns.isin(['periodType', 'currencyCode'])]
        data_y = data_y.melt(id_vars = ['symbol', 'asOfDate'])
        data_y = data_y.replace([np.nan], None)
        data_y['freq'] = 'y'
        data_y.columns = ['ticker', 'date', 'account', 'value', 'freq']
        
        
        data_q = data.all_financial_data(frequency = 'q')
        data_q.reset_index(inplace = True)
        data_q = data_q.loc[:, ~data_q.columns.isin(['periodType', 'currencyCode'])]
        data_q = data_q.melt(id_vars = ['symbol', 'asOfDate'])
        data_q = data_q.replace([np.nan], None)
        data_q['freq'] = 'q'
        data_q.columns = ['ticker', 'date', 'account', 'value', 'freq']
        
        data_fs = pd.concat([data_y, data_q], axis=0)

        args = data_fs.values.tolist()
        mycursor.executemany(query_fs, args)
        con.commit()

    except:

        print(ticker)
        error_list.append(ticker)

    time.sleep(2)

engine.dispose()
con.close()

 18%|█▊        | 17/96 [01:48<08:48,  6.69s/it]

BRKa


 19%|█▉        | 18/96 [01:51<07:34,  5.83s/it]

BRKb


100%|██████████| 96/96 [10:05<00:00,  6.30s/it]
